In [2]:
elementsData = dict()
elementsChars = ['H', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'mu', 'sigma', 'epsil']
elements = ['AlCl', 'AlCl2', 'AlCl3', 'GaCl', 'GaCl2', 'GaCl3', 'NH3', 'H2', 'HCl', 'N2', 'Al', 'Ga', 'AlN', 'GaN']
dataFile = open('data.txt', 'r')
lines = dataFile.readlines()
for line in lines:
    dataArr = line.rstrip().split(';')
    elementsData[dataArr[0]] = dict()
    for i in range(0, len(dataArr) - 1):
        elementsData[dataArr[0]][elementsChars[i]] = float(dataArr[i + 1])

R = 8.314  # Дж/(кмоль⋅К) - универсальная газовая постоянная
P = 100000  # Па – полное лав

'''
d - коэффициент диффузии (м2/сек)
compIndex - 'AlCl', 'GaN, ... 
temperature - температура в К
pressure - полное давление в системе (P_i_g - P_i_e)
'''
def dGasCoef(compIndex, temperature, pressure):
    numerator = 2.628 * 10 ** (-2) * temperature ** (3 / 2)

    dataForIndex = elementsData[compIndex]
    dataForN2 = elementsData['N2']

    sigma = (dataForIndex['sigma'] + dataForN2['sigma']) / 2
    epsil = (dataForIndex['epsil'] * dataForN2['epsil']) ** (1 / 2)
    mu = (2 * dataForIndex['mu'] * dataForN2['mu']) / (dataForIndex['mu'] + dataForN2['mu'])
    omega = 1.074 * (temperature / epsil) ** (-0.1604)

    return numerator / (pressure * sigma * omega * mu ** (1 / 2))


In [3]:
print(dGasCoef('AlCl', 370, 9000))

0.0008936897676922723
